In [42]:
import pandas as pd
import numpy as np
#!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import os


Using pyppteer, scrape the ra.co/events/de/berlin website which shows the coming 7 days events.
-scrape the event link then change the scrape loop to scrape each event link for the artists and artist info page link, the venue, the date and ticket availabilty. 
-scrape each artist info link for the artist real name, and the links the artist has given too their material
-extract the sound cloud link
-using the sound cloud api extract the genre for the top 15 songs on the artists sound cloud and create a dictonary with the artist and the genres
-train a  

In [3]:
data = pd.read_csv('../Party_recommender/pypetteer/myfile.txt', sep=" ", header=None)
data[['0', 'djs']] = data[0].str.split("dj/", 1, expand=True)

data

,0,0,djs
0,'https://ra.co/dj/gitteverfuehrt','https://ra.co/,gitteverfuehrt'
1,'https://ra.co/dj/sabinehoffmann','https://ra.co/,sabinehoffmann'
2,'https://ra.co/dj/annina','https://ra.co/,annina'
3,'https://ra.co/dj/misterwillis','https://ra.co/,misterwillis'
4,'https://ra.co/dj/estherdune','https://ra.co/,estherdune'
...,...,...,...
147,'https://ra.co/dj/saramiller','https://ra.co/,saramiller'
148,'https://ra.co/dj/marcelfengler','https://ra.co/,marcelfengler'
149,'https://ra.co/dj/soundstream','https://ra.co/,soundstream'
150,'https://ra.co/dj/mtvare','https://ra.co/,mtvare'


In [4]:
artist_list=(data['djs'])
djs=pd.DataFrame(artist_list.str.rstrip("'"))
djs=djs.drop_duplicates(keep='first')
djs=djs.reset_index(drop=True)
djs

,djs
0,gitteverfuehrt
1,sabinehoffmann
2,annina
3,misterwillis
4,estherdune
...,...
95,saramiller
96,marcelfengler
97,soundstream
98,mtvare


Now we seach spotify for the artists

In [5]:
#first activate the API credentials

username = 'Nathan-Austin'
client_id = "ab96bcae5de243ae84f28bc07473386a"
client_secret = "11ebd032938d4494a3aced507c40e31d"
redirect_uri= "http://localhost:7777/callback"
scope = 'user-top-read'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token )

In [6]:
#here we search the dj names extracted from the RA scrape. 
artist_ids={}
for idx, item in enumerate(djs['djs']):
    result= sp.search(q=item, type= 'artist', limit=3) #limit of 3 returns 3 results per name search, this needs to be filterd
    artist_ids[idx] =result

artist_ids

{0: {'artists': {'href': 'https://api.spotify.com/v1/search?query=gitteverfuehrt&type=artist&offset=0&limit=3',
   'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3QryVD03gGZOLQQXjy3EoA'},
     'followers': {'href': None, 'total': 302899},
     'genres': ['brostep', 'dark trap', 'edm', 'electronic trap', 'filthstep'],
     'href': 'https://api.spotify.com/v1/artists/3QryVD03gGZOLQQXjy3EoA',
     'id': '3QryVD03gGZOLQQXjy3EoA',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab6761610000e5eb82504d534c072b1d72e38a8c',
       'width': 640},
      {'height': 320,
       'url': 'https://i.scdn.co/image/ab6761610000517482504d534c072b1d72e38a8c',
       'width': 320},
      {'height': 160,
       'url': 'https://i.scdn.co/image/ab6761610000f17882504d534c072b1d72e38a8c',
       'width': 160}],
     'name': 'Getter',
     'popularity': 59,
     'type': 'artist',
     'uri': 'spotify:artist:3QryVD03gGZOLQQXjy3EoA'},
    {'external_urls': {'spotify'

Lets save the output of the search to a json file

In [7]:
import json
with open('event_data/artists_2.json','w') as f:
    json.dump(artist_ids, f)



In [8]:
len(artist_ids)

100

Here we must get all the useful data from the API call such as the artist name, id number, and genres linked to the artist.

In [36]:
artist_id =[]
genre=[]
artist_name=[]
for i in range(len(artist_ids)):
    items = artist_ids[i].items()
    for feature in (items):
        #print(feature[1]['items'][0]['uri'])
        id_ =feature[1]['items'][0]['uri']
        artist_id.append(id_)
        genres=feature[1]['items'][0]['genres']
        genre.append(genres)
        name=feature[1]['items'][0]['name']
        artist_name.append(name)
        
        #print(artist_id)
        #print(genre)
        #print(artist_name)

IndexError: list index out of range

In [48]:
search_df=pd.DataFrame(artist_id, index=artist_name, columns=['artist_id'])
search_df['genres']=genre
search_df

,artist_id,genres
Getter,spotify:artist:3QryVD03gGZOLQQXjy3EoA,"[brostep, dark trap, edm, electronic trap, fil..."
Armin Hoffmann,spotify:artist:1u3MrkLceIr9dnJxsIARBh,[]
Annina,spotify:artist:42Tt0c612ANogSbFJXaqUJ,[]
Mr. Williamz,spotify:artist:3iL52u3kr14P6cTpB0VuEs,"[modern reggae, uk dancehall]"
Esther Phillips,spotify:artist:0WZ7IgzdjPvwFdjDjjuZm7,"[classic soul, funk, jazz blues, rhythm and bl..."
...,...,...
Klex,spotify:artist:3DwyaR8SVRCHRY2YusGlVP,[]
Kalipo,spotify:artist:7ud6dY3K4gi4Q0uVlLd8Hi,[german house]
Tülay Maciran,spotify:artist:4j6jsMeLNhbGsukg8ofn92,[]
Kangding Ray,spotify:artist:20UWNE4rEU7YMO0GHq4F26,"[dub techno, glitch, intelligent dance music, ..."


In [ ]:
type(artist_name)

list

In [10]:

df= pd.DataFrame(genre, columns = ['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10'])
df

,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
0,None,None,None,None,None,None,None,None,None,None
1,dub techno,float house,minimal dub,outsider house,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None
4,orchestral soundtrack,soundtrack,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
171,bleep techno,experimental house,None,None,None,None,None,None,None,None
172,dance pop,edm,electro house,german dance,pop,pop dance,tropical house,None,None,None
173,bluegrass fiddle,None,None,None,None,None,None,None,None,None
174,None,None,None,None,None,None,None,None,None,None


In [11]:
df = df.drop(df.index[0])
df=df.reset_index(drop=True)
df

,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
0,dub techno,float house,minimal dub,outsider house,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None
3,orchestral soundtrack,soundtrack,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
170,bleep techno,experimental house,None,None,None,None,None,None,None,None
171,dance pop,edm,electro house,german dance,pop,pop dance,tropical house,None,None,None
172,bluegrass fiddle,None,None,None,None,None,None,None,None,None
173,None,None,None,None,None,None,None,None,None,None


In [ ]:

df.isna().sum()

genre1      95
genre2     126
genre3     153
genre4     163
genre5     168
genre6     170
genre7     170
genre8     173
genre9     174
genre10    174
dtype: int64

In [12]:
df =df.drop(['genre6','genre7','genre8', 'genre9', 'genre10'], axis =1)
df


,genre1,genre2,genre3,genre4,genre5
0,dub techno,float house,minimal dub,outsider house,None
1,None,None,None,None,None
2,None,None,None,None,None
3,orchestral soundtrack,soundtrack,None,None,None
4,None,None,None,None,None
...,...,...,...,...,...
170,bleep techno,experimental house,None,None,None
171,dance pop,edm,electro house,german dance,pop
172,bluegrass fiddle,None,None,None,None
173,None,None,None,None,None


In [13]:
df.fillna(value=pd.np.nan, inplace=True) 
df

/tmp/ipykernel_10294/1889086677.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.fillna(value=pd.np.nan, inplace=True)


,genre1,genre2,genre3,genre4,genre5
0,dub techno,float house,minimal dub,outsider house,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,orchestral soundtrack,soundtrack,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
170,bleep techno,experimental house,NaN,NaN,NaN
171,dance pop,edm,electro house,german dance,pop
172,bluegrass fiddle,NaN,NaN,NaN,NaN
173,NaN,NaN,NaN,NaN,NaN


Lets get all the unique genres extracted from our list. Some will be wrong becasue we have the wrong artist but it will help with the recommender

In [14]:
genres_list= pd.unique(df[['genre1','genre2','genre3', 'genre4', 'genre5']].values.ravel('K'))

Lets fill a column as a test

In [15]:
df['genres6']= df['genre1'].fillna(pd.Series(np.random.choice(genres_list, size= len(df.index))))
df

,genre1,genre2,genre3,genre4,genre5,genres6
0,dub techno,float house,minimal dub,outsider house,NaN,dub techno
1,NaN,NaN,NaN,NaN,NaN,german dance
2,NaN,NaN,NaN,NaN,NaN,preschool children's music
3,orchestral soundtrack,soundtrack,NaN,NaN,NaN,orchestral soundtrack
4,NaN,NaN,NaN,NaN,NaN,deep tropical house
...,...,...,...,...,...,...
170,bleep techno,experimental house,NaN,NaN,NaN,bleep techno
171,dance pop,edm,electro house,german dance,pop,dance pop
172,bluegrass fiddle,NaN,NaN,NaN,NaN,bluegrass fiddle
173,NaN,NaN,NaN,NaN,NaN,georgian pop


In [17]:
djs['genres']= df['genres6'].fillna(pd.Series(np.random.choice(genres_list, size=len(djs.index))))
djs['genres']= djs['genres'].fillna(pd.Series(np.random.choice(genres_list, size=len(djs.index))))
djs['genres']= djs['genres'].fillna(pd.Series(np.random.choice(genres_list, size=len(djs.index))))
djs

,djs,genres
0,lilith-ge,dub techno
1,vril,german dance
2,lysas,preschool children's music
3,elios,orchestral soundtrack
4,jamesnewmarch,deep tropical house
...,...,...
566,oliverrosemann,italo dance
567,inverseelement,french indietronica
568,monoloc,swedish pop rap
569,frankieflowerz,georgian electronic


In [16]:
djs.isnull().sum()


djs    0
dtype: int64

Now lets add some locations using a similar method. First we need 100 location names?

In [18]:
clubs = pd.read_csv('nightclubs.txt', sep=",", header=None)
clubs

,0
0,40seconds Club
1,Abspannwerk Buchhändlerhof
2,Adagio Berlin
3,Arena Berlin
4,Astra Kulturhaus
...,...
56,Weekend Club
57,Wilde Renate
58,WMF Club
59,YAAM (club)


In [19]:
clubs_list =pd.unique(clubs.values.ravel('K'))
clubs_list

array([' 40seconds Club', ' Abspannwerk Buchhändlerhof', ' Adagio Berlin',
       ' Arena Berlin', ' Astra Kulturhaus', ' Badehaus Berlin',
       ' Ballhaus Spandau', ' Bang Bang Club', ' Bar 25',
       ' Bassy Cowboy Club', ' Berghain', ' Big Eden', ' Bunker (Berlin)',
       ' Cassiopeia (Berlin)', ' Club der Visionäre', ' Cookies Berlin',
       ' Die Busche', ' Eimer (Berlin)', ' Felix ClubRestaurant',
       ' Geburtstagsklub', ' Goya (club)', ' Hafenbar', ' Hoppetosse',
       ' Kater Blau', ' Kater Holzig', ' Kiki Blofeld', ' KitKatClub',
       ' Klub der Republik ', ' Knaack club', ' Kumpelnest 3000',
       ' L.U.X', ' La Belle (Diskothek)', ' Leydicke', ' Lido Berlin',
       ' Linientreu', ' M-BIA', ' Magnet Club', ' Maria am Ostbahnhof',
       ' Matrix (club)', ' Miami (club)', ' Ostgut', ' Picknick Club',
       ' Prince Charles Berlin', ' Privatclub (Berlin)', ' Q-Dorf',
       ' Quasimodo Charlottenburg', ' RAW Tempel', ' Ritter Butzke',
       ' Sage Club', ' SchwuZ

In [20]:
djs['clubs'] = clubs[0].fillna(pd.Series(np.random.choice(clubs_list, size=len(djs.index))))
djs['clubs'] = djs['clubs'].fillna(pd.Series(np.random.choice(clubs_list, size=len(djs.index))))
djs

,djs,genres,clubs
0,lilith-ge,dub techno,40seconds Club
1,vril,german dance,Abspannwerk Buchhändlerhof
2,lysas,preschool children's music,Adagio Berlin
3,elios,orchestral soundtrack,Arena Berlin
4,jamesnewmarch,deep tropical house,Astra Kulturhaus
...,...,...,...
566,oliverrosemann,italo dance,Prince Charles Berlin
567,inverseelement,french indietronica,Club der Visionäre
568,monoloc,swedish pop rap,Badehaus Berlin
569,frankieflowerz,georgian electronic,Abspannwerk Buchhändlerhof


In [ ]:
len(djs['clubs'])


571

In [21]:
by_clubs=djs.sort_values(["clubs"])

In [22]:
by_clubs =by_clubs.set_index('clubs')

In [24]:
by_clubs.to_csv('event_data')